<a href="https://colab.research.google.com/github/karenbennis/BigData/blob/master/Vine_table_anlaysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Download a Postgres driver that will allow Spark to interact with Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-06-13 20:49:16--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  3.60MB/s    in 0.2s    

2020-06-13 20:49:17 (3.60 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ChallengeETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Configure settings for RDS
jdbc_url="jdbc:postgresql://dataviz.cigpctobxd5d.us-east-2.rds.amazonaws.com:5432/"
config = {"user":"postgres", 
          "password": "jack1257", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Read in vine_table from Postgres
vine_df = spark.read.jdbc(jdbc_url, "vine_table", properties=config)
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RY01SAV7HZ8QO|          4|            0|          0|   N|
|R1XX8SDGJ4MZ4L|          5|            0|          0|   N|
|R149Q3B5L33NN5|          5|            0|          0|   N|
|R2ZVD69Z6KPJ4O|          1|            0|          2|   N|
|R1DIKG2G33ZLNP|          4|            0|          0|   N|
|R3L6FGKAW0EYFI|          1|            1|          1|   N|
| RAO0QZH5VC6VI|          1|            0|          0|   N|
|R25IK0UAHWNB22|          3|            0|          0|   N|
|R2A9IHKZMTMAL1|          5|            1|          2|   N|
| R5XVKTHL6SITI|          5|            0|          0|   N|
|R2QZZOSTDDY1IE|          3|            3|          6|   N|
|R38CUDCFPSNYTD|          5|            0|          0|   N|
| RM6ZR6NH052YH|          3|            1|          2|   N|
| RUQK5N4WH8UN8|          5|            

# **Analysis of vine_df (aka vine_table)**

In [ ]:
# Make separate DataFrames for reviews based on whether or not the review is a vine review

# Filter table for rows where review is a vine review
vine_review_df = vine_df.filter(vine_df["vine"] == 'Y')
# Order by helpful votes
vine_review_df= vine_review_df.orderBy(vine_review_df['helpful_votes'].desc())
# Show the vine_review_df
vine_review_df.show()
# Count number of rows in vine_review_df
vine_review_df.count()
# Get summary stats for vine reviews
vine_review_stats = vine_review_df.describe()
vine_review_stats.show()

# Filter table for rows where review is not a vine review
non_vine_review_df = vine_df.filter(vine_df["vine"] == 'N')
# Order by helpful votes
non_vine_review_df= non_vine_review_df.orderBy(non_vine_review_df['helpful_votes'].desc())
# Show the non_vine_review_df
non_vine_review_df.show()
# Count number of rows in non_vine_review_df
non_vine_review_df.count()
# Get summary stats for non-vine reviews
non_vine_review_stats = non_vine_review_df.describe()
non_vine_review_stats.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2JYDQZPWVTH7R|          4|         1751|       1861|   Y|
| RZ48DAKK0QPDX|          4|          656|        694|   Y|
|R2ST3UKU0JFLR6|          5|          586|        606|   Y|
|R2L0G0XRXWBY2X|          3|          494|        520|   Y|
|R34F7I0N94G97C|          5|          446|        472|   Y|
|R2OFR7E2RI196K|          5|          388|        449|   Y|
| RGWV3QLJQXU9X|          5|          332|        380|   Y|
|R122J9QGFCZTR0|          5|          324|        342|   Y|
|R38RLM92AHWLPF|          5|          284|        301|   Y|
|R2H24YTRLARCZJ|          4|          277|        297|   Y|
|R1OSNVALVQIZEK|          4|          277|        297|   Y|
|R13ESDYL0P5EYC|          4|          272|        313|   Y|
| RQF84KLBP0RHF|          5|          272|        303|   Y|
| RD0E1RZSSYGGN|          5|          26

## **Direct comparison of percentage of 5-star reviews (vine_review_df vs non_vine_review_df)**

### Vine reviews

In [ ]:
# Get percentage of 5-star reviews: Vine
vine_five_star_count = vine_review_df.filter(vine_review_df['star_rating']==5).count()
vine_count = 2106
vine_five_star_percent = vine_five_star_count / vine_count * 100
print(vine_five_star_count)
print(vine_five_star_percent)

854
40.55080721747388


### Non-Vine reviews

In [ ]:
# Get percentage of 5-star reviews: Non-Vine
non_vine_five_star_count = non_vine_review_df.filter(non_vine_review_df['star_rating']==5).count()
nonvine_count = 703744
non_vine_five_star_percent = non_vine_five_star_count / nonvine_count * 100
print(non_vine_five_star_count)
print(non_vine_five_star_percent)

373107
53.01743247544562


Based on percentage, it seems that non-vine reviewers give 5-star ratings more often than the vine reviewers; however, the number of vine reviewers is substancially smaller. Given the difference in number of observations, it would be of greater value to run t-tests to see if it can be concluded that vine reviewers are similar to to regular reviewers. The next section includes this analysis.

## **Run t tests for helpful_votes and star_rating to see if vine data is biased**
This will be calculated somewhat manually (using values obtained from the summary statistics for each table and manual calculations) since there is no quick method to run a t-test in pySpark.

In [ ]:
from scipy import stats
import math

###**t test for means of helpful_reviews from vine_review_df and non_vine_review_df**
Null Hypothesis - The average number of helpful votes from the vine_review_df data is equal to that of the non_vine_review_df

Alternate Hypothesis - The average number of helpful votes from the vine_review_df data is not equal to that of the non_vine_review_df

In [ ]:
# Perform t test for means of helpful_votes from vine_review_df and non_vine_review_df

# Get values from summary stats dfs:
avg_helpful_votes_vine = 11.934947768281102
stddev_helpful_votes_vine = 53.570847213619054
vine_count = 2106

avg_helpful_votes_nonvine = 3.5863495816660604
stddev_helpful_votes_nonvine = 37.37964346362367
nonvine_count = 703744

# Calculate t-value: t_value = (mean(X1) - mean(X2)) / sqrt((std1 / sqrt(n1))^2 + (std2 / sqrt(n2))^2)
t_value = (avg_helpful_votes_vine - avg_helpful_votes_nonvine) / math.sqrt((stddev_helpful_votes_vine / math.sqrt(vine_count))**2 + (stddev_helpful_votes_nonvine / math.sqrt(nonvine_count))**2)

# Calculate degrees of freedom: degrees of freedom = n1 + n2 - 2
deg_freedom = vine_count + nonvine_count -2

# Calculate p-value: p_value = (1 - stats.t.cdf(abs(t_value), deg_freedom)) * 2
p_value = (1 - stats.t.cdf(abs(t_value), deg_freedom)) * 2
p_value

8.906209103543006e-13

With a p-value of 8.91e-13, there is sufficient evidence to reject the null hypothesis at a 95% confidence interval. Therefore the mean helpful_votes of the vine_reviews is statistically different from the nonvine_review_df helpful_votes mean. This implies that the vine_review_df data is not representative of the population since the means are different and the p-value is low enough to be confident that these results were not due to chance.

### **t test for means of star_rating from vine_review_df and non_vine_review_df**
Null Hypothesis - The average star rating from the vine_review_df data is equal to that of the nonvine_review_df

Alternate Hypothesis - The average star rating from the vine_review_df data is not equal to that of the nonvine_review_df

In [ ]:
# Perform t test for means of star_rating from vine_review_df and non_vine_review_df

# Get values from summary stats dfs:
avg_star_rating_vine = 4.073599240265907
stddev_star_rating_vine = 0.9862696623724604
vine_count = 2106

avg_star_rating_nonvine = 3.901600582029829
stddev_star_rating_nonvine = 1.4562449004745772
nonvine_count = 703744

# Calculate t-value: t_value = (mean(X1) - mean(X2)) / sqrt((std1 / sqrt(n1))^2 + (std2 / sqrt(n2))^2)
t_value = (avg_star_rating_vine - avg_star_rating_nonvine) / math.sqrt((stddev_star_rating_vine / math.sqrt(vine_count))**2 + (stddev_star_rating_nonvine / math.sqrt(nonvine_count))**2)

# Calculate degrees of freedom: degrees of freedom = n1 + n2 - 2
deg_freedom = vine_count + nonvine_count -2

# Calculate p-value: p_value = (1 - stats.t.cdf(abs(t_value), deg_freedom)) * 2
p_value = (1 - stats.t.cdf(abs(t_value), deg_freedom)) * 2
p_value

1.5543122344752192e-15

With a p-value of 1.55e-15, there is sufficient evidence to reject the null hypothesis at a 95% confidence interval. Therefore the mean star_rating of the vine_reviews is statistically different from the nonvine_review_df star_rating mean. This implies that the vine_review_df data is not representative of the population since the means are different and the p-value is low enough to be confident that these results were not due to chance.